## Pipeline to erase any object in an image using segmentation (Yolo v8), correction (LabelMe) and inpainting (LaMa)

> #### Pre-requisite : Mention parent folder path
> File Structure should be structured as below when running without Yolov8 text labels:
>> parent_folder
>>> images
>
> File Structure should be structured as below when running with Yolov8 text labels:
>> parent_folder
>>> images<br>
>>> labels
>
> **Only Cell 1 should be edited for initializing parent folder, initialization of sub-folders path is not required.**
>
> #### Step 1 : Run Auto-Annotation using Yolo v8 *yolov8x-seg.pt* segmentation model
>
> - Yolov8 labels are in text format and the text label format is [Class] [x1,y1] [x2,y2] [x3,y3] ... where x and y are normalized from 0 to 1 irrespective of image height and width for flexibility.
> - LabelMe requires json format for labels, so to edit the annotation or to view in LabelMe, Yolov8 text labels are converted to Json labels.
> - Since Yolov8 is very accurate in segmenting an object, it segments every small details which results in huge number of segment points * (x,y), which makes it hard to edit in LabelMe. *Ramer-Douglas-Peucker algorithm* is applied while converting from text to json labels to reduce the points in the polygons. **So, the number of points in text labels are not equal to the number of points in json labels.**
> - Unlike LabelImg, LabelMe requires both images and labels in one folder, the script is designed to combine everything in one folder with right naming and format.
> - By the end of Step 1 i.e, Cell 8, we will be having .png images and .json labels in one folder called *LabelMe* to edit in LabelMe application.
>
> #### Step 2 : Annotation and Correction in LabelMe
>
> - Click for [how to install LabelMe](https://github.com/labelmeai/labelme#installation)
> - Make sure to uncheck **File > Save with Image Data** whenever the application is launched. **It resets everytime LabelMe is closed.** 
> 
> #### Step 3 : Post-processing of corrected images and labels to run on LaMa
>
> - After Annotation correction in LabelMe, the scripts in Step 3 must be executed before feeding it to LaMa.
> - LaMa requires image and image mask both in .png format. 
> - Naming is as follows -
>>   image1.png<br>
>>   image1_mask001.png
> - All the images and image masks must be in one folder.
>
> #### Step 4 : Installation of LaMa on virtual environment and how to run 
>
> - Guide at the end of this notebook.

### Pre-requisite : Initializing parent folder

> *Enter the path to parent folder below.*

In [13]:
# Cell 1

# Define the path to the parent folder
parent_folder = r"/path/to/parent_folder"  # Replace with the actual path to parent folder

> *Initializes and creates all the required sub-folders.*

In [ ]:
# Cell 2

import os

img_folder = os.path.join(parent_folder, "images")               # Folder with images

# List of folder names to be created in parent_folder
folder_names = [
    'labels',
    'png_images',
    'json_labels',
    'LabelMe',
    'png_mask',
    'LaMa'
]

# Create folders and assign them to variables
txt_folder, png_folder, json_folder, output_dir, pngmask_folder, LaMa_folder = [None] * 6

for i, folder_name in enumerate(folder_names):
    folder_path = os.path.join(parent_folder, folder_name)
    try:
        os.makedirs(folder_path, exist_ok=True)
        print(f"Folder '{folder_name}' created successfully!")
        
        # Assign folder paths to variables
        if i == 0: txt_folder = folder_path
        elif i == 1: png_folder = folder_path
        elif i == 2: json_folder = folder_path
        elif i == 3: output_dir = folder_path
        elif i == 4: pngmask_folder = folder_path
        elif i == 5: LaMa_folder = folder_path

    except FileExistsError:
        if i == 0: txt_folder = folder_path
        print(f"Folder '{folder_name}' already exists.")
    except Exception as e:
        print(f"An error occurred while creating folder '{folder_name}': {e}")

# Print the folder paths assigned to variables
print(f"Folder1: {txt_folder}")
print(f"Folder2: {png_folder}")
print(f"Folder3: {json_folder}")
print(f"Folder4: {output_dir}")
print(f"Folder5: {pngmask_folder}")
print(f"Folder6: {LaMa_folder}")

### Step 1 : Yolov8 Instance Segmentation

> *Ignore Cell 3 if already cloned*

In [ ]:
# Cell 3

!git clone https://github.com/ultralytics/ultralytics
!cd ultralytics

> *Yolov8 predicts and creates extra labels folder in the parent directory. (Use save=False to avoid storing inference images)*

In [ ]:
# Cell 4

import os
import shutil

from ultralytics import YOLO

yolo_output = os.path.join(parent_folder, "yolo_output")

# Load the model
model = YOLO("yolov8x-seg.pt")

# Use the model with stream=True to avoid caching
results = model(img_folder, save=True, save_txt=True, classes=[0], show_boxes=False, stream=True, project=yolo_output)

# Iterate over the streaming results
for result in results:
    # Process each result individually
    print(result)

# Define the path of the output directory
current_dir_path = os.path.join(yolo_output, "predict\labels")

# Get the destination path three level up
destination_path = os.path.abspath(os.path.join(current_dir_path, '../../../'))  # This takes you to Y

# Define the full destination path with the directory name
destination_dir_path = os.path.join(destination_path, os.path.basename(current_dir_path))

# Copy the directory to the destination
try:
    shutil.copytree(current_dir_path, destination_dir_path, dirs_exist_ok=True)
    print(f"Directory '{current_dir_path}' copied to '{destination_dir_path}' successfully!")
except Exception as e:
    print(f"An error occurred while copying the directory: {e}")

> *Ignore Cell 5 if already installed*

In [ ]:
# Cell 5

# For Image Processing
!pip install pillow

# Contains Ramer-Douglas-Peucker (RDP) algorithm to apply on polygons to reduce points
!pip install pillow simplification

> *Cell 6 Converts Yolo v8 '0 to 1' normalized coordinates **text files** to LabelMe readable **json files***

In [ ]:
# Cell 6

#Packages
import os
import json
from PIL import Image
from simplification.cutil import simplify_coords

# Helper function to convert YOLO txt format to JSON format for Labelme
def yolo_txt_to_points(yolo_data, img_width, img_height):
    objects = []
    lines = yolo_data.splitlines()

    for line in lines:
        data = line.split()
        class_id = int(data[0])
        coords = list(map(float, data[1:]))

        # Convert normalized coordinates(0 to 1) from Yolo v8 to image dimensions(width, height)
        points = []
        for i in range(0, len(coords), 2):
            x = coords[i] * img_width
            y = coords[i+1] * img_height
            points.append([x, y])

        objects.append({"class_id": class_id, "points": points})

    return objects

# Simplify polygon points using the Ramer-Douglas-Peucker algorithm
# This function is used to reduce points from Yolo v8 Auto-Annotation to simplify annotation check on Labelme
def simplify_polygon(points, tolerance=2.0):
    simplified_points = simplify_coords(points, tolerance)
    return simplified_points

# This function is designed to structure json file as per Labelme json format
def yolo_to_labelme_json(yolo_data, img_width, img_height, img_path, label_map, tolerance=2.0):
    objects = yolo_txt_to_points(yolo_data, img_width, img_height)
    
    shapes = []
    for obj in objects:
        class_id = obj['class_id']
        points = obj['points']
        
        # Simplify points using RDP algorithm as mentioned above
        simplified_points = simplify_polygon(points, tolerance)

        # Json Header
        labelme_shape = {
            "label": label_map[class_id],
            "points": simplified_points,
            "group_id": None,
            "description": "",
            "shape_type": "polygon",
            "flags": {},
            "mask": None
        }
        shapes.append(labelme_shape)

    # Json Footer
    labelme_format = {
        "version": "5.5.0",
        "flags": {},
        "shapes": shapes,
        "imagePath": img_path,
        "imageData": None,
        "imageHeight": img_height,
        "imageWidth": img_width
    }
    
    return labelme_format

# Batch conversion function
def batch_convert_yolo_to_json(txt_folder, img_folder, json_folder, label_map, tolerance=2.0):
    if not os.path.exists(json_folder):
        os.makedirs(json_folder)
    
    # Loop through each txt file in the folder
    for txt_file in os.listdir(txt_folder):
        if txt_file.endswith(".txt"):
            # Read YOLO txt file
            with open(os.path.join(txt_folder, txt_file), 'r') as file:
                yolo_data = file.read().strip()
            
            # Generate image file name from txt file name
            img_file = os.path.splitext(txt_file)[0] + ".jpg"
            img_path = os.path.join(img_folder, img_file)
            
            # Get image dimensions
            with Image.open(img_path) as img:
                img_width, img_height = img.size

            # Convert to Label Studio JSON format with simplification
            converted_data = yolo_to_labelme_json(yolo_data, img_width, img_height, img_path, label_map, tolerance)
            
            # Output JSON file name
            output_file = os.path.join(json_folder, os.path.splitext(txt_file)[0] + ".json")
            
            # Save JSON
            with open(output_file, 'w') as json_file:
                json.dump(converted_data, json_file, indent=4)

    print(f"Conversion complete. JSON files are saved in {json_folder}.")

label_map = {0: "person", 1: "class", 2: "class"}               # Mapping of class IDs to labels
tolerance = 5.0                                                 # Tolerance for reducing polygon points, Higher value = fewer points

# Perform batch conversion with point reduction and multiple objects
batch_convert_yolo_to_json(txt_folder, img_folder, json_folder, label_map, tolerance)

> *Cell 7 converts all the images which are in .jpg format to .png format and stores in png_images folder*

In [ ]:
# Cell 7

import os
import shutil
from PIL import Image

# Convert JPG images to PNG and save them in the output folder
for filename in os.listdir(img_folder):
    if filename.lower().endswith('.jpg') or filename.lower().endswith('.jpeg'):
        # Open the image file
        img = Image.open(os.path.join(img_folder, filename))

        # Convert and save as PNG with the same name
        base_filename = os.path.splitext(filename)[0]
        img.save(os.path.join(png_folder, base_filename + '.png'), 'PNG')

print("Conversion and file transfer complete!")

> *Cell 8 copies converted .png files and .json files to LabelMe folder*

In [ ]:
# Cell 8

# Function to copy contents from source to destination
def copy_contents(source, destination):
    try:
        for item in os.listdir(source):
            src_path = os.path.join(source, item)
            dest_path = os.path.join(destination, item)
            
            # Copy directories recursively
            if os.path.isdir(src_path):
                shutil.copytree(src_path, dest_path, dirs_exist_ok=True)
            else:
                # Copy files
                shutil.copy2(src_path, dest_path)
        print(f"Contents of '{source}' copied to '{destination}' successfully!")
    except Exception as e:
        print(f"An error occurred while copying contents from '{source}' to '{destination}': {e}")

# Copy contents of png_images to LabelMe folder
copy_contents(png_folder, output_dir)

# Copy contents of json_labels to LabelMe folder
copy_contents(json_folder, output_dir)

### Step 2 : LabelMe

> *Segment annotation correction and check.*<br>
> *Do not execute step 3 without segment corrections as erasing an object takes different approach in mask boundaries.*

### Step 3 : Post-processing of corrected images and labels to run on LaMa

> *Cell 9 does JSON to PNG mask conversion (filename.json to filename_mask001.png) and saves it in png_mask folder*

In [ ]:
# Cell 9

import os
import json
import numpy as np
from PIL import Image, ImageDraw

def json_to_png(json_file, output_file):
    with open(json_file, 'r') as f:
        data = json.load(f)

    shapes = data.get('shapes', [])
    image_width = data.get('imageWidth', 1920)
    image_height = data.get('imageHeight', 1080)

    # Create a new blank mask image
    mask = Image.new('L', (image_width, image_height), 0)
    draw = ImageDraw.Draw(mask)

    for shape in shapes:
        if shape['shape_type'] == 'polygon':
            points = shape.get('points', [])
            if points:
                # Convert points to integer tuples
                polygon = [(int(p[0]), int(p[1])) for p in points]
                draw.polygon(polygon, outline=1, fill=255)

    # Save the mask as PNG
    mask.save(output_file)

def process_folder(json_folder, pngmask_folder):
    # Ensure the output folder exists
    os.makedirs(pngmask_folder, exist_ok=True)
    
    # Loop through all files in the folder
    for filename in os.listdir(json_folder):
        if filename.endswith('.json'):
            json_file = os.path.join(json_folder, filename)
            # Create output file name with suffix "_mask001.png"
            base_name = os.path.splitext(filename)[0]  # Get the base name (without extension)
            output_file = os.path.join(pngmask_folder, f"{base_name}_mask001.png")
            json_to_png(json_file, output_file)
            print(f"Processed: {filename}")

# batch operation
process_folder(output_dir, pngmask_folder)

> *Cell 10 moves inference_images.png and inference_images_mask001.png to LaMa folder for LaMa to execute*

In [ ]:
# Cell 10

import os
import shutil
from PIL import Image

# Create the output directory if it doesn't exist
os.makedirs(LaMa_folder, exist_ok=True)

# Convert JPG images to PNG and save them in the output folder
for filename in os.listdir(img_folder):
    if filename.lower().endswith('.jpg') or filename.lower().endswith('.jpeg'):
        # Open the image file
        img = Image.open(os.path.join(img_folder, filename))

        # Convert and save as PNG with the same name
        base_filename = os.path.splitext(filename)[0]
        img.save(os.path.join(LaMa_folder, base_filename + '.png'), 'PNG')

# Move JSON files from pngmask_folder to LaMa_folder
for filename in os.listdir(pngmask_folder):
    if filename.lower().endswith('.png'):
        # Move the JSON file to the output directory
        shutil.copy(os.path.join(pngmask_folder, filename), os.path.join(LaMa_folder, filename))

print("Conversion and file transfer complete!")

> *Cell 11 deletes all the unnecessary data which is not required to run LaMa. (Do not execute Cell 11 before confirming the data in LaMa folder)* 

In [ ]:
# .png images
shutil.rmtree(png_folder)

# .json labels
shutil.rmtree(json_folder)

# .png mask images
shutil.rmtree(pngmask_folder)

### Step 4 : Installation of LaMa on virtual environment and how to run 

#### LaMa Installation Guide (only tested on conda env)

> #### Python version
>> **Python 3.10.14 (Tested and Recommended)** 
> 
> #### LaMa Repository
>> git clone https://github.com/advimman/lama.git<br>
>> cd lama
> 
> #### Pytorch Installation
> **Pytorch for GPU 12.4**<br>
> conda install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124<br>
>
> **Pytorch for GPU 10.2 (Recommended)**<br>
> conda install pytorch torchvision torchaudio cudatoolkit=10.2 -c pytorch -y
> 
> #### Edited requirements to resolve version conflicts
> - pip install -r requirements.txt *(please copy this if the installation of recommended requirements was failed)*
>> pyyaml<br>
>> tqdm<br>
>> numpy<br>
>> easydict==1.9.0<br>
>> scikit-image<br>
>> scikit-learn<br>
>> opencv-python<br>
>> tensorflow<br>
>> joblib<br>
>> matplotlib<br>
>> pandas<br>
>> albumentations==0.5.2<br>
>> hydra-core==1.1.0<br>
>> pytorch-lightning==1.2.9<br>
>> tabulate<br>
>> kornia==0.5.0<br>
>> webdataset<br>
>> packaging<br>
>> wldhx.yadisk-direct<br>
> 
> #### Install Detectron2
>> git clone https://github.com/facebookresearch/detectron2.git<br>
>> python -m pip install -e detectron2
> 
> #### Temporary env variables
>> set TORCH_HOME=%CD%<br>
>> set PYTHONPATH=%CD%
> 
> #### Download and unzip model
>> curl -LJO https://huggingface.co/smartywu/big-lama/resolve/main/big-lama.zip <br>
>> tar -xf big-lama.zip
> 
> #### Prediction *(Runs on both CPU and GPU)*
>
>> *(Use this everytime when using new prompt)*<br>
>> set TORCH_HOME=%CD%<br>
>> set PYTHONPATH=%CD%
> 
>> python bin/predict.py model.path="enter\your\model\path\here" indir="enter\your\testdata\path\here" outdir="enter\your\output\path\here"
> 
> #### Refined Prediction *(Only runs on GPU)*
> 
>> python bin/predict.py refine=True model.path="enter\your\model\path\here" indir="enter\your\testdata\path\here" outdir="enter\your\output\path\here"